# The JEC Game
## a.k.a let's do a little math!

Let's now try to calculate the JEC for some jets by hand. We can then compare these JEC to those produced by the FactorizedJetCorrector (JEC's truely on-the-fly)!

We'll start by doing the usual setup:

In [ ]:
# Loads the ROOT environment and style
import ROOT as r
from collections import OrderedDict
from Analysis.JMEDAS.tdrstyle_mod14 import *
from tqdm import tqdm_notebook
import math,os
#from ipywidgets import FloatProgress
#from IPython.display import display

# Set the ROOT style
r.gROOT.Macro("rootlogon.C")
setTDRStyle()

Next well define a list of jets and their 4-momenta and some properties of the event/jets.

Note: This list was obtained from the file JECNtuple_MiniAOD.root using the command:
```
t->Scan("rho:jtpt*jtjec[][0].second:jtpt*jtjec[][1].second:jtpt:jteta:jtarea","Iteration$<1","colsize=15 precision=12")```
<details>
<summary><font color='blue'>Show resulting table...</font></summary><pre>
***********************************************************************************************************************************
*    Row   * Instance *             rho * jtpt*jtjec[][0] * jtpt*jtjec[][1] *            jtpt *           jteta *          jtarea *
***********************************************************************************************************************************
*        0 *        0 *   27.9735126495 *    25.268358248 *   18.9651246116 *   21.4697628021 * -0.942416906357 *  0.498665511608 *
*        1 *        0 *   22.3101711273 *   43.5569762444 *   40.8701305967 *   46.3169059753 *   1.53576695919 *   0.52858543396 *
*        2 *        0 *   8.20386505127 *   5498.01234685 *   5497.16506556 *   5722.63330078 * 0.0332580916584 *  0.468745559454 *
*        3 *        0 *   27.1829166412 *   580.947463703 *   577.381168944 *   624.002685547 * -0.851783692837 *  0.518612146378 *
*        4 *        0 *   20.8539142609 *   5283.47962611 *   5278.49402182 *   5565.54394531 * -0.033908639103 *   0.52858543396 *
*        5 *        0 *   9.71122264862 *   6089.61327727 *   6087.97697711 *   6310.93359375 *  0.132899135351 *   0.52858543396 *
*        6 *        0 *   9.27750873566 *    6279.8396654 *   6278.37107391 *   6619.79492188 * -0.067324548959 *  0.518612146378 *
*        7 *        0 *   7.43301820755 *   5343.21748126 *   5342.55496417 *   5538.21240234 *  0.101400665939 *  0.508638799191 *
*        8 *        0 *   21.0998935699 *   6100.72340942 *   6095.37590476 *   6381.86376953 * -0.101955212653 *  0.468745559454 *
*        9 *        0 *   16.7489337921 *   263.715815175 *   260.994576252 *   278.435638428 *  0.771255135536 *  0.548532068729 *
*       10 *        0 *   23.4246501923 *   112.452929425 *   109.869716363 *   123.661773682 *  -1.27514147758 *  0.458772271872 *
*       11 *        0 *    15.866063118 *   5406.69153684 *   5403.19707086 *   5624.81152344 * 0.0114215835929 *   0.52858543396 *
*       12 *        0 *   13.0399713516 *   6174.34569195 *   6171.32095584 *   6424.43994141 *  0.034766767174 *  0.548532068729 *
*       13 *        0 *    20.933637619 *    5884.5125568 *   5879.08260911 *   6120.21533203 * 0.0492748767138 *  0.508638799191 *
*       14 *        0 *   5.30949354172 *   6099.07277047 *   6099.09964008 *   6349.25683594 * 0.0703366398811 *  0.498665511608 *
*       15 *        0 *   27.7770862579 *   6217.47270042 *   6209.08945881 *   6500.92236328 * -0.151779085398 *  0.508638799191 *
*       16 *        0 *   29.8529777527 *    41.852376492 *   35.0890546832 *    37.819984436 * 0.0481784641743 *  0.498665511608 *
*       17 *        0 *   22.2874279022 *   5640.52473148 *   5634.93013144 *   5928.61376953 * -0.196134760976 *  0.508638799191 *
*       18 *        0 *   8.46105861664 *   6376.88622617 *   6375.68510997 *   6609.17871094 * 0.0966885611415 *  0.518612146378 *
*       19 *        0 *   25.3683223724 *    2449.8855947 *   2446.26350129 *   2575.81567383 *  0.318729907274 *  0.498665511608 *
*       20 *        0 *  0.785045206547 *   24.1153980231 *   24.3696350618 *   28.2773895264 *   3.83928585052 *  0.498665511608 *
*       21 *        0 *   31.6383686066 *    5840.0275811 *   5831.54656377 *   6135.47753906 * -0.176622077823 *  0.478718876839 *
*       22 *        0 *   11.6279621124 *   6060.82858257 *   6058.70824967 *      6280.59375 *  0.105112791061 *  0.478718876839 *
*       23 *        0 *   34.1100692749 *   5931.00007673 *   5920.49350123 *   6163.32470703 *   0.01558414381 *   0.52858543396 *
*       24 *        0 *   29.3820381165 *   6284.89322321 *   6276.39133184 *   6617.70751953 * -0.070473849773 *  0.488692194223 *
</pre>
</details>

In [ ]:
jet_list = [
    {'rho':27.9735126495,'pt':25.268358248,'eta':-0.942416906357,'area':0.498665511608},
    {'rho':22.3101711273,'pt':43.5569762444,'eta':1.53576695919,'area':0.52858543396},
    {'rho':8.20386505127,'pt':5498.01234685,'eta':0.0332580916584,'area':0.468745559454},
    {'rho':27.1829166412,'pt':580.947463703,'eta':-0.851783692837,'area':0.518612146378},
]

We will be making use of the JEC files in [../data/JECs/Spring16_25nsV5_MC/](../data/JECs/Spring16_25nsV5_MC/)

*Note*: You can us ```%alias``` to see a list of built-in Jupyter "magic" commands 

### L1FastJet

In [ ]:
%cat ../data/JECs/Summer16_23Sep2016V4_MC/Summer16_23Sep2016V4_MC_L1FastJet_AK4PFchs.txt

<font color='red'>Compute the L1FastJet JEC by hand and then find the corrected p<sub>T</sub></font>

<details><summary><font color='blue'>Show example code for jet 1...</font></summary>
<p>
    
```python
import math
jec = 1.0-(0.4986655*(-1.82028+(0.728067*27.972512)*(1.+(-0.0897926)*math.log(25.268358)))/25.268358)
print jec
print 25.268358*jec 
```

</p>
</details>

<details><summary><font color='blue'>Show TF3 version...</font></summary>
<p>

```python
f = r.TF3("f","1.0-z*([0]+([1]*x)*(1.0+[2]*log(y)))/y")
f.SetParameters(-1.82028,0.728067,-0.0897926)
jec = f.Eval(27.972512,25.268358,0.4986655)
print jec
print 25.268358*jec
```

</p>
</details>

<details><summary><font color='blue'>Show loop version...</font></summary>
<p>
    
```python
jet_list_parameters = [
    [-1.82028,0.728067,-0.0897926],
    [-1.32373,0.899498,-0.180374],
    [-1.88478,0.663812,-0.0590435],
    [-1.97421,0.747076,-0.0886402],
]
f = r.TF3("f","1.0-z*([0]+([1]*x)*(1.0+[2]*log(y)))/y")
for j, jet in enumerate(jet_list):
    f.SetParameters(jet_list_parameters[j][0],jet_list_parameters[j][1],jet_list_parameters[j][2])
    print jet['pt']*f.Eval(jet['rho'],jet['pt'],jet['area'])
```
    
</p>
</details>

In [ ]:
# Do your work here

<font color='red'>Make a list of jets with your newly calculated JEC. This will aid in the next stage of the computation.</font>

<details><summary><font color='blue'>Solution...</font></summary>
<p>

```python
jet_list_l1 = [
    {'rho':27.972512,'pt':18.965124217,'eta':-0.942416,'area':0.4986655},
    {'rho':22.310171,'pt':40.8701325024,'eta':1.5357669,'area':0.5285854},
    {'rho':8.2038650,'pt':5497.64115121,'eta':0.0332580,'area':0.4687455},
    {'rho':27.182916,'pt':577.381173741,'eta':-0.851783,'area':0.5186121},
]
```

</p>
</details>

### L2Relative & L3Absolute

In [ ]:
%cat ../data/JECs/Summer16_23Sep2016V4_MC/Summer16_23Sep2016V4_MC_L2Relative_AK4PFchs.txt

<font color='red'>Do the same computation but with the L2Relative/L3Absolute corrections. Remember to chain the corrections together. Make a table of the corrected jets. Compare this to the corrected jets from the ntuple.</font>
<details><summary><font color='blue'>Hint...</font></summary>
The the new p<sub>T</sub><sup>L1</sup> is now the input p<sub>T</sub> for this computation.
</details>
<details><summary><font color='blue'>Solution (code)...</font></summary>
<p>

```python
jet_list_l1 = [
    {'rho':27.972512,'pt':18.965124217,'eta':-0.942416,'area':0.4986655},
    {'rho':22.310171,'pt':40.8701325024,'eta':1.5357669,'area':0.5285854},
    {'rho':8.2038650,'pt':5497.64115121,'eta':0.0332580,'area':0.4687455},
    {'rho':27.182916,'pt':577.381173741,'eta':-0.851783,'area':0.5186121},
]
jet_list_parameters = [
    [1.15158,16.1036,-0.00601581,-0.00127766,0.00034833],
    [1.13994,37.285,-0.00122072,-0.000286474,3.01379e-05],
    [1.04627,2570.31,3.02494e-06,-7.73772e-08,9.74598e-11],
    [1.07988,432.787,1.01127e-05,2.32598e-08,-3.57916e-10],
]
f = r.TF3("f","max(0.0001,[0]+((min(3000.,x)-[1])*([2]+((min(3000.,x)-[1])*([3]+((min(3000.,x)-[1])*[4]))))))")
for j, jet in enumerate(jet_list_l1):
    f.SetParameters(jet_list_parameters[j][0],jet_list_parameters[j][1],jet_list_parameters[j][2],jet_list_parameters[j][3],jet_list_parameters[j][4])
    print jet['pt']*f.Eval(jet['pt'])
```

</p>
</details>

<details><summary><font color='blue'>Solution (table)...</font></summary>
<p>

```python
jet_list_l1l2l3 = [
    {'rho':27.972512,'pt':21.4697622893,'eta':-0.942416,'area':0.4986655},
    {'rho':22.310171,'pt':46.3169040985,'eta':1.5357669,'area':0.5285854},
    {'rho':8.2038650,'pt':5723.12884379,'eta':0.0332580,'area':0.4687455},
    {'rho':27.182916,'pt':624.002697501,'eta':-0.851783,'area':0.5186121},
]
```

</p>
</details>

In [ ]:
# Do your work here

### FactorizedJetCorrector to the rescue!

This process would be horrible if we had to do this by hand! Luckily CMSSW has us covered. Next we will compare our "by-hand" answers to those produced by the FactorizedJetCorrector.

The code is set up to compare the result of applying the L1FastJet corrections. To check the full L1L2L3 correction chain try uncommenting the other jet corrections.

In [ ]:
era = "Summer16_23Sep2016V4_MC"
vPar = r.vector(r.JetCorrectorParameters)()
vPar.push_back(r.JetCorrectorParameters(os.path.expandvars('$CMSSW_BASE/src/Analysis/JMEDAS/data/JECs/{}/{}_L1FastJet_AK4PFchs.txt'.format(era,era))))
#vPar.push_back(r.JetCorrectorParameters(os.path.expandvars('$CMSSW_BASE/src/Analysis/JMEDAS/data/JECs/{}/{}_L2Relative_AK4PFchs.txt'.format(era,era))))
#vPar.push_back(r.JetCorrectorParameters(os.path.expandvars('$CMSSW_BASE/src/Analysis/JMEDAS/data/JECs/{}/{}_L3Absolute_AK4PFchs.txt'.format(era,era))))
jec = r.FactorizedJetCorrector( vPar )
for jet in jet_list:
    jec.setJetEta( jet['eta'] )
    jec.setJetPt ( jet['pt'] )
    #jec.setJetE  ( jet['energy'] )
    jec.setJetA  ( jet['area'] )
    jec.setRho   ( jet['rho'] )
    #jec.setNPV   ( jet['npv'] )
    print jet['pt']*jec.getCorrection()

The same process works for the uncertainties, but the computation is slightly different.

In [ ]:
jet_list_l1l2l3 = [
    {'rho':27.972512,'pt':,'eta':-0.942416,'area':0.4986655},
    {'rho':22.310171,'pt':,'eta':1.5357669,'area':0.5285854},
    {'rho':8.2038650,'pt':,'eta':0.0332580,'area':0.4687455},
    {'rho':27.182916,'pt':,'eta':-0.851783,'area':0.5186121},
]
era = "Summer16_23Sep2016V4_MC"
jecUnc = r.JetCorrectionUncertainty(os.path.expandvars('$CMSSW_BASE/src/Analysis/JMEDAS/data/JECs/{}/{}_Uncertainty_AK4PFchs.txt'.format(era,era)))
for jet in jet_list_l1l2l3:
    ptCor = jet['pt']
    jecUnc.setJetEta( jet['eta'] )
    #jecUnc.setJetPhi( jet['phi'] )
    jecUnc.setJetPt( ptCor )
    ptCor_shiftedUp = ptCor*(1+jecUnc.getUncertainty(1))
    jecUnc.setJetEta( jet['eta'] )
    #jecUnc.setJetPhi( jet['phi'] )
    jecUnc.setJetPt( ptCor )
    ptCor_shiftedDown = ptCor*(1+(-1*jecUnc.getUncertainty(0)))
    print str(ptCor),'+',str(abs(ptCor-ptCor_shiftedUp)),'-',str(abs(ptCor-ptCor_shiftedDown))